# Introduction to Pywr-DRB
## Overview:

This tutorial will guide you through linking the `PywrDRB-ML` plug-in to predict water temperature at Lordville (TempLSTM) and the salt front location (SalinityLSTM), two critical variables for thermal mitigation efforts aimed at protecting cold-water fish and for salt front repulsion to safeguard Philadelphia's drinking water intake.

To activate these features, users must first download the [`PywrDRB-ML` plug-in](https://github.com/philip928lin/PywrDRB-ML).

Currently, TempLSTM and SalinityLSTM have the simulation capacity shown below.

| ML model        | From        | To         |
|-----------------|-------------|------------|
| TempLSTM        | 7/28/2006   | 12/31/2023 | 
| SalinityLSTM    | 10/1/1963   | 12/31/2023 | 


# Step 1: Make model with TempLSTM and SalinityLSTM

In [ ]:
import os
import pywrdrb
wd = "your working directory here"
PywrDRB_ML_plugin_path = "PywrDRB_ML directory"

inflow_type = 'pub_nhmv10_BC_withObsScaled'
model_filename = os.path.join(wd, f"{inflow_type}.json")
output_filename = os.path.join(wd, f"{inflow_type}.hdf5")

# TempLSTM settings
temp_options = {
    "PywrDRB_ML_plugin_path": PywrDRB_ML_plugin_path, 
    "start_date": None,                     # If other start date is desired, set here
    "activate_thermal_control": False,      # If True, will activate thermal control. This feature is not yet available.
    "Q_C_lstm_var_name": "QbcTavg_Q_C",     # LSTM variable name for Q_C flow (01425000)
    "Q_i_lstm_var_name": "QbcTavg_T_C",     # LSTM variable name for Q_i flow (01417000 + Lordville natural inflow)
    "disable_tqdm": False                   # If True, will disable tqdm progress bars
    }

# SalinityLSTM settings
salinity_options = {
    "PywrDRB_ML_plugin_path": PywrDRB_ML_plugin_path, 
    "start_date": None,                             # If other start date is desired, set here
    "Q_Trenton_lstm_var_name": "01463500_bc",       # LSTM variable name for Q_Trenton flow (01463500)
    "Q_Schuylkill_lstm_var_name": "01474500_bc",    # LSTM variable name for Q_Schuylkill flow (01474500)
    "disable_tqdm": False                           # If True, will disable tqdm progress bars   
    }

mb = pywrdrb.ModelBuilder(
    inflow_type=inflow_type, 
    start_date="2006-01-01",
    end_date="1970-12-31",
    options={
        "temperature_model": temp_options,
        "salinity_model": salinity_options,
        }
    )

mb.make_model()
mb.write_model(model_filename)

# Step 2: Run model

In [ ]:
model = pywrdrb.Model.load(str(model_filename))
recorder = pywrdrb.OutputRecorder(
    model=model,
    output_filename=output_filename,
    parameters=[p for p in model.parameters if p.name]
)
stats = model.run()

# Step 3: Load TempLSTM and SalinityLSTM related ouputs

In [ ]:
data = pywrdrb.Data()
results_sets = [
    'temperature', 
    'salinity', 
    ]
data.load_output(output_filenames=[output_filename], results_sets=results_sets)

df_temperature = data.temperature[inflow_type][0]
df_salinity = data.salinity[inflow_type][0]

Any output values that equals to `-99` is not an error. It represents no value should be outputed from that parameter.

In [ ]:
df_temperature.head(5)

In [ ]:
df_salinity.haead(5)